## Graphs & Graph Algorithms

**Theory/definitions:**


- <b>Undirected graph.</b> In a directed graph the edges have no direction associated with them. For example, a social network of friendships between people or existence of a wiring connection between electronic components on a circuit board. <br><br>

- <b>Directed graph.</b> In a directed graph, relationships between vertices have a direction associated with them, such as order of execution in scheduling problems (perform task 1 before task 2) or traffic flow direction in a (potentially one way) street. <br><br>
    
- <b>Bipartite graph.</b> A bipartite graph $G=(V,E)$, is a graph vertex set, $V$, of which can be partitioned into two nonempty subsets $A$ and $B$ such that each edge of $G$ has one endpoint in $A$ and one endpoint in $B$. Bipartite graphs often arise in applications involving matching problems, such as assigning tasks to workers or allocating resources to projects.<br><br>
    
- <b>Strongly connected component.</b> A strongly connected component of a directed graph, $G=(V,E)$, is a maximal set of vertices, $C\subseteq V$, such that for every pair of vertices, $u$ and $v$ in $C$, we have both $u\leadsto v$ and $v \leadsto u$. Strongly connected components are often used as a tool for understanding the structure and connectivity of directed graphs in various real-world applications (e.g. social network analysis).


#### BFS

Given a graph $G=(V, E)$, and a **source** vertex $S$:
* Discover every vertex that is reachable from $S$, and queue all vertexs one layer down

Uses a single FIFO queue containing some vertices at a distance k, possibly followed by some vertices at distance k+1. 

A good example of a computational problem for which BFS could be preferred is finding the shortest path between two nodes in an unweighted graph. If the shortest path is significantly shorter than the possible longest simple path (ie. path without loops) then BFS would terminate much earlier before all nodes are explored. Note if DFS was used for such a problem, it could also find a path from a start node to a desired destination note, however one would not know if it is the shortest possible path. <br><br>
    

#### DFS

Choose a first undiscovered vertex and explore neighbors recursively until no unexplored neighbors are left. For each vertex $v$, once all out going edges are discovered, the algorithm must backtrack to vertex from which $v$ was discovered.

Undiscovered vertices are coloured `white`, those with fully explored neighbors `black`, and the rest `gray`.

An example of a computational problem that is well-suited for DFS is finding a path between two nodes in a maze or labyrinth with obstacles. In this case, DFS can be used to explore all possible paths in the maze and find a solution which is not necessarily the most optimal one.

#### Topological Sort

A Topological sort of a directed acycled graph (DAG) is a linear ordering of its vertices such that an edge $(u, v)$ requires $u$ appear before $v$ in the ordering.

If the graph contains a cycle this is of course impossible.

#### MWST

Generic MST:
* $A$ is the empty set
* while $A$ does not form an MST
* find an edge that is safe and add it to $A$
* An edge is a light edge crossing a cut if its weight is the minimum of those that cross

Hence any *light edge* is safe to be appended to $A$ Greedily

##### -> Kruskal

##### -> Prim's

### Shortest Path From Source

##### -> Bellman-Ford

##### -> Dijkstra

#### Floyd-Warshall

Let $d_{ij}^k$ be the weight of the shortest path from $i$ to $j$ where all intermediate vertices can be numbered $[1..k]$ i.e. there are a maximum of $k$ vertices on the path. Hence $d_{ij}^{(0)} = w_{ij}$.

$d_{ij}^k =  \begin{cases} w_{ij}  & \text{if } k = 0 \\ 
                         min[ d_{ij}^{k-1} , d_{ik}^{k-1} + d_{kj}^{k-1}] & \text{if } k \geq 1 \end{cases}$

### Max-Flow Min-Cut

For a given flow $f$, the net flow across any cut is the same, and it equals $|f|$.

A minimum cut of a network is a cut whose *capacity* is minimum over all cuts of the network

flow counts edges going in both directions across the cut, but capacity counts only edges going from the source side of the cut toward the sink side.

If $f$ is a flow in flow network $G$ with source $s$, and sink $t$, then the following conditions are equivalent:
* $f$ is a maximum flow of $G$
* The residual network contains no augmenting paths.
* For some cut of $G$ $(S, T)$, $|f| = c(S, T)$

A minimum cut of a network is a cut whose capacity is minimum over all cuts of the
network

### Maximum bipartite matching

A *matching* is a subset of edges $M \in E$ such that every vertex has at most one incident edge in M .

Let $G$ be a bipartite graph subsets $L$ and $R$, and let $G'$ bea corresponding flow network.

If $M$ is a *matching* in $G$
* There is an integer flow $f$ in $G'$ with value $|f| = |M|$

If there is an integer flow $f$ in $G'$
* There is a matching with value $|f| = |M|$

Hence The cardinality of a maximum matching in bipartite graph equals the value of maximum flow in the corresponding network.


In [84]:
from pathlib import Path
import pygraphviz as pgv
import copy
import random
from IPython.display import HTML, display

# Directed Edge from a to b
class Edge():
    def __init__(self, a, b, w) -> None:
        self.a = a
        self.b = b
        self.w = w
    
    def __repr__(self) -> str:
        return f"{self.a}->{self.b} [{self.w}]"
    
    def T(self):
        return Edge(self.b, self.a, self.w)
    
## Vertex, with string ID 'v'
class Vertex():
    def __init__(self, id):
        self.id = id
        # defaults 
        self.color = "white"   
        self.d = float('inf')   # Distance Metric
        self.pi = None
        self.s = 0              # Start 
        self.f = 0              # Finish

    # comparisons allow object or object.v
    def __contains__(self, item):
        if isinstance(item, Vertex):
            return item.id == self.id
        return item == self.id
    
    def __eq__(self, item):
        if isinstance(item, Vertex):
            return item.id == self.id
        return item == self.id
    
    def __repr__(self) -> str:
        if self.pi is not None:
            return f"{self.id} d:[{self.d}] s[{self.s}] f[{self.f}] parent[{self.pi.id}]"
        else:
            return f"{self.id} d:[{self.d}] s[{self.s}] f[{self.f}]"

    
# Top level graph object
class Graph():
    # Initialise with
    # v: list of vertex ID's
    # e: list of edge tuples (a, b, weight)
    # V: List of Vertex Elements
    # E: List of Edge Elements
    def __init__(self, v: list, e: list) -> None:
        self.V = []
        self.E = []
        for q in v:
            self.V.append(Vertex(q))
        for t in e:
            self.E.append(Edge(t[0], t[1], t[2]))
        # construct adjacency matrix/dict
        self.build_dict()

    def build_dict(self):
        self.Aj = dict()
        # allocate the primary dict
        for vert in self.V:
            self.Aj[vert.id] = dict()
        ## fill it in
        for edge in self.E:
            self.Aj[edge.a][edge.b] = edge.w

    def make_undirected(self):
        for i in range(len(self.E)):
            self.E.append(Edge(self.E[i].b, self.E[i].a, self.E[i].w))
        # update adjacency matrix/dict
        self.build_dict()

    def id_to_V(self, a) -> Vertex:
        # finds the vertex object with ID a
        return self.V[self.V.index(a)]

    def __repr__(self) -> str:
        return str(self.Aj)

    def g_viz_bfs(self, dest_path: Path):

        def render_node_descent(vertex, pgv_graph):
            text = f"{vertex.id} d:{vertex.d}"
            pgv_graph.add_node(vertex.id, shape="rectangle", style="filled", fillcolor="#fcf0cf", label=text)
            
            if vertex.pi is not None:
                pgv_graph.add_edge(vertex.pi.id, vertex.id)

        # Create a new Graph object
        pgv_graph = pgv.AGraph(directed=False)
        # sort the vertices into ascending order and get min
        Q = copy.deepcopy(self.V)
        Q = sorted(Q, key=lambda x: x.d, reverse=False)
        for v in Q:
            render_node_descent(v, pgv_graph=pgv_graph)

        print(f"Saving visualisation to {dest_path}")
        pgv_graph.draw(dest_path, prog="dot")

    def g_viz_dfs(self, dest_path: Path):

        def render_node_descent(vertex, pgv_graph):
            text = f"{vertex.id} s:{vertex.s} f:{vertex.f}"
            pgv_graph.add_node(vertex.id, shape="rectangle", style="filled", fillcolor="#fcf0cf", label=text)
            
            if vertex.pi is not None:
                pgv_graph.add_edge(vertex.pi.id, vertex.id)

        # Create a new Graph object
        pgv_graph = pgv.AGraph(directed=False)
        # sort the vertices into ascending order and get min
        Q = copy.deepcopy(self.V)
        Q = sorted(Q, key=lambda x: x.d, reverse=False)
        for v in Q:
            render_node_descent(v, pgv_graph=pgv_graph)

        print(f"Saving visualisation to {dest_path}")
        pgv_graph.draw(dest_path, prog="dot")

### BFS

Given a graph $G=(V, E)$, and a **source** vertex $S$:
* Discover every vertex that is reachable from $S$, and queue all vertexs one layer down

Uses a single FIFO queue containing some vertices at a distance k, possibly followed by some vertices at distance k+1. 

<img src="../media/bfstest.png" border="0" class="bg-primary" width="400px"> 



In [85]:
_V = ["s","a","b","c","d","e","f","g"]
_E = [("s", "a", 1), ("s", "b", 1), ("a", "d", 1), ("b", "e", 1), ("d", "e", 1), ("d", "g", 1), \
      ("e", "d", 1), ("e", "c", 1), ("c", "f", 1)]

G = Graph(_V, _E)

# Takes a graph object, and the source vertex object to start from
def bfs(G: Graph, s: Vertex):
    for v in G.V:
        v.color = "white"
        v.d = float('inf') # distance
        v.pi = None # parent
    s.color = "gray"
    s.d = 0

    # BFS maintains a FIFO for vertices on the discovery frontier to search
    # Start with the source vertex
    Q = []; Q.append(s)

    while len(Q):
        # while there are vertices to search, grab the first one
        # and loop through the edges belonging to it
        u = Q.pop(0)
        # for each vertex in the adj matrix directed to from u
        for vert_id in G.Aj[u.id]:
            vert = G.id_to_V(vert_id)
            # if we haven't discovered it yet, add it to the frontier
            if (vert.color == "white"):
                vert.color = "gray"
                vert.d = u.d + 1 
                vert.pi = u
                Q.append(vert) # queue the vertex to be discovered next time
        u.color = "black" 

bfs(G, G.id_to_V("s"))

for v in G.V:
    print(v)

# Anti-Caching Visualiser
c_ = random.randint(0,2e9)
G.g_viz_bfs(f"{c_}.png")
display(HTML(f'<img src="{c_}.png" border="0" class="bg-primary" width="150px">'))

s d:[0] s[0] f[0]
a d:[1] s[0] f[0] parent[s]
b d:[1] s[0] f[0] parent[s]
c d:[3] s[0] f[0] parent[e]
d d:[2] s[0] f[0] parent[a]
e d:[2] s[0] f[0] parent[b]
f d:[4] s[0] f[0] parent[c]
g d:[3] s[0] f[0] parent[d]
Saving visualisation to 729420098.png


### DFS

Choose a first undiscovered vertex and explore neighbors recursively until no unexplored neighbors are left. For each vertex $v$, once all out going edges are discovered, the algorithm must backtrack to vertex from which $v$ was discovered.

Undiscovered vertices are coloured `white`, those with fully explored neighbors `black`, and the rest `gray`.

In [86]:
_V = ["s","a","b","c","d","e","f","g"]
_E= [("s", "b", 1), ("s", "c", 1), ("a", "s", 1), ("a", "b", 1), \
    ("b", "c", 1), ("b", "d", 1), ("c", "s", 1), ("d", "a", 1), \
    ("e", "b", 1), ("f", "c", 1), ("f", "e", 1), ("f", "g", 1), \
    ("g", "d", 1), ("g", "e", 1), ("g", "f", 1)]

G = Graph(_V, _E)

# recursive DFS visitation
def DFS_visit(G: Graph, u: Vertex, time: int):
    time += 1
    u.s = time # record start time
    u.color = "gray" # set as on frontier

    # loop through all vertices connected to u
    for v_id in G.Aj[u.id]:
        vert = G.id_to_V(v_id)
        # if one is undiscovered, discover it and descend
        if (vert.color == "white"):
            vert.pi = u
            time = DFS_visit(G, vert, time)

    u.color = "black" # set as complete
    time += 1
    u.f = time # set finish time
    return time

# 
def dfs(G: Graph, s: Vertex):
    time = 0
    for v in G.V:
        v.color = "white"
        v.s = 0
        v.f = 0
        v.pi = None
    
    # Visit any disjoint undiscovered sections of the graph
    for v in G.V:
        if v.color == "white":
            time = DFS_visit(G, v, time)
    return time

dfs(G, G.id_to_V("s"))

for v in G.V:
    print(v)

# Anti-Caching Visualiser
c_ = random.randint(0,2e9)
G.g_viz_dfs(f"{c_}.png")
display(HTML(f'<img src="{c_}.png" border="0" class="bg-primary" width="450px">'))

s d:[inf] s[1] f[10]
a d:[inf] s[6] f[7] parent[d]
b d:[inf] s[2] f[9] parent[s]
c d:[inf] s[3] f[4] parent[b]
d d:[inf] s[5] f[8] parent[b]
e d:[inf] s[11] f[12]
f d:[inf] s[13] f[16]
g d:[inf] s[14] f[15] parent[f]
Saving visualisation to 1971076232.png


### Topological Sort

A Topological sort of a directed acycled graph (DAG) is a linear ordering of its vertices such that an edge $(u, v)$ requires $u$ appear before $v$ in the ordering.

Works by essentially just listing items in reverse finishing order.

If the graph contains a cycle this is of course impossible.

In [87]:
_V = ["s","a","b","c","d","e","f","g"]
_E= [("s", "b", 1), ("s", "c", 1), ("b", "c", 1), ("b", "d", 1), \
    ("d", "a", 1),  ("e", "b", 1),  ("f", "c", 1),  ("f", "e", 1),  \
    ("f", "g", 1),  ("g", "d", 1),  ("g", "e", 1)]
G = Graph(_V, _E)

def toposort(G: Graph):
    final_time = dfs(G, G.id_to_V("s"))
    # create a list for each finishing time
    A = [-1 for _ in range(final_time+1)]
    # index and load A by finishing time
    for v in G.V:
        A[v.f] = v
    # remove all -1's (not updates indices)
    A = list(filter(lambda a: a != -1, A))
    A.reverse() # reverse finishing time
    return A

A = toposort(G)
print( " , ".join(x.id for x in A))

f , g , e , s , b , d , a , c


### Strongly Connected Components

A strongly connected set of a directed graph is a maximal set of vertices such that for every set of vertices, there is a path from one to the other and the other to one.

* Call `DFS` to compute the finishing times `f` for every node
* Compute the graph's transpose $G^T$
* Call DFS $G^T$ but in the main loop consider in order of decreasing `f`
* Output the vertices in the depth-first forest as separately strongly connected components.

In [88]:
## TODO!!

### MWST

**Prior to each iteration, A is a subset of a MST**

Generic MST:
* $A$ is the empty set
* while $A$ does not form an MST
* find an edge that is safe and add it to $A$
* An edge is a light edge crossing a cut if its weight is the minimum of those that cross

Hence any *light edge* is safe to be appended to $A$

In [113]:
_V = ["s","a","b","c","d","e","f","g"]
_E= [("s", "a", 2), ("s", "c", 1),  ("a", "d", 4),  ("a", "b", 2),  ("b", "c", 2), \
    ("b", "f", 6), ("b", "e", 3), ("e", "f", 1), ("f", "g", 2), ("d", "e", 1), \
    ("e", "f", 1)]
G = Graph(_V, _E)
G.make_undirected()

### KRUSKAL
# Create a growing collection of multiple forests
# Find a safe edge to add to the forest by 
#   finding all edges that connect any two trees and choosing the light edge.
# Automatically ends when all edges are accessible in a single forest.

# get the index that contains the node x
def get_tree_containing_x(tree_list: list, x: str) -> int:
    for i, l in enumerate(tree_list):
        if x in l:
            return i
        
def mst_kruskal(G: Graph):
    # MST vertex list
    A = []

    # make a deep copy of the vertex list where each is a tree (list of vertices)
    forest_set = []
    for v in G.V:
        forest_set.append([Vertex(v.id)])

    # sort the edges into ascending order by weight
    min_E = sorted(G.E, key=lambda x: x.w, reverse=False)
    
    for edge in min_E:
        # if the nodes of the edge are in two different trees
        # edge is safe, add it to the forest
        forest_a = get_tree_containing_x(forest_set, edge.a)
        forest_b = get_tree_containing_x(forest_set, edge.b)
        if forest_a != forest_b:
            A.append(edge)
            # A.append(edge.T()) # ?explicitly bidirected
            # combine the two trees
            newset = forest_set.pop( get_tree_containing_x(forest_set, edge.a) )
            newset += forest_set.pop( get_tree_containing_x(forest_set, edge.b) )
            forest_set.append(newset)
    return A

A = mst_kruskal(G)
print(A)

[s->c [1], e->f [1], d->e [1], s->a [2], a->b [2], f->g [2], b->e [3]]


In [118]:
G = Graph(_V, _E)
G.make_undirected()

## Prim's Algorithm
# from an arbitrary root s, add light edges to A that connect A 
#   to an isolated vertex

def mst_prim(G: Graph, s: Vertex):
    # deep copy of the vertex list
    # Q must represent a min priority queue, where the 
    #   key is the minimum weight connecting the vertex to any
    #   vertex already in the tree
    Q = []
    for v in G.V:
        Q.append(Vertex(v.id))
    Q = sorted(Q, key=lambda x: x.s, reverse=False)

    # use v.s as the key element
    for v in Q:
        v.s = float('inf')
        v.pi = None

    # ensure S is processed first
    Q[Q.index("s")].s = 0
    
    A = []
    while len(Q):
        # Get the next node to add to the tree
        u = Q.pop(0)
        
        # copy its edge to the tree
        if u.pi is not None:
            A.append(Edge(u.id, u.pi.id, u.s))
    
        # Update the min p queue Q
        # loop through all edges that are connected and in Q 
        for v_id in G.Aj[u.id]: 
            if v_id in Q:
                v = Q[Q.index(v_id)]
                w = G.Aj[u.id][v_id]
                if w < v.s and v in Q:
                    v.s = w
                    v.pi = u

            Q = sorted(Q, key=lambda x: x.s, reverse=False)

    return A


print(mst_prim(G, G.id_to_V("s")))


[c->s [1], a->s [2], b->c [2], e->b [3], f->e [1], d->e [1], g->f [2]]


### Shortest Path - Bellman-Ford

Progressive edge relaxation, decreasing an estimate for $v.d$ for each $v$ until the true $\delta$ is reached.

The algorithm will return true if and only if there are no reachable negative weight cycles

Runtime: $\Theta(VE)$

In [123]:
_V = ["s","a","b","c","d","e"]
_E= [("s", "a", 5), ("a", "c", 5),  ("c", "d", 1),  ("d", "e", 4),  ("e", "b", 5), \
    ("b", "s", 2), ("b", "a", 1), ("a", "e", 2), ("e", "c", 2)]
G = Graph(_V, _E)
G.make_undirected()

def bellman_ford(G: Graph, s: Vertex):
    # configure every node with inf d and no parent
    for v in G.V:
        v.d = float('inf')
        v.pi = None
    # s is the source node
    s.d = 0

    # for i in the range of every vertex
    for i in range(len(G.V)):
        # note that we have a separate edge instance for each direction 
        # for every edge in the graph
        #   if the directed edge reduces the distance between the two nodes
        #   set the new distance and update the parent
        for e in G.E:
            a = G.id_to_V(e.a)
            b = G.id_to_V(e.b)
            # relax
            if b.d > a.d + e.w:
                b.d = a.d + e.w
                b.pi = a

    # !!check for negative acyclics!!
    # i.e failures
    for e in G.E:
            a = G.id_to_V(e.a)
            b = G.id_to_V(e.b)
            if b.d > a.d + e.w:
                raise Exception
            
    return True

bellman_ford(G, G.id_to_V("s"))
for v in G.V:
    print(v)

# Anti-Caching Visualiser
c_ = random.randint(0,2e9)
G.g_viz_bfs(f"{c_}.png")
display(HTML(f'<img src="{c_}.png" border="0" class="bg-primary" width="100px">'))

s d:[0] s[0] f[0]
a d:[3] s[0] f[0] parent[b]
b d:[2] s[0] f[0] parent[s]
c d:[7] s[0] f[0] parent[e]
d d:[8] s[0] f[0] parent[c]
e d:[5] s[0] f[0] parent[a]
Saving visualisation to 295090127.png


### Dijkstra

Maintains a set of vertices $S$ whose final $v.d$ have been computed. Repeatedly select a non-final vertex with the lowest estimated $v.d$, adds it to $S$ and relaxes all edges that are connected to the vertex.

Use a min-priority queue for optimal next vertex selection.

Only functions with *non negative weights*


In [138]:
_V = ["s","a","b","c","d","e"]
_E= [("s", "a", 5), ("a", "c", 5),  ("c", "d", 1),  ("d", "e", 4),  ("e", "b", 5), \
    ("b", "s", 2), ("b", "a", 1), ("a", "e", 2), ("e", "c", 2)]
G = Graph(_V, _E)
G.make_undirected()

def dijkstra(G: Graph, s: Vertex):
    # configure every node with inf d and no parent
    for v in G.V:
        v.d = float('inf')
        v.pi = None
    # s is the source node
    s.d = 0

    # set of fully computed vertices
    S = []

    # Local minp Q
    # deep copy vertex state
    Q = copy.deepcopy(G.V)

    # This implementation uses the real vertices as the source and storage of all parameters
    # and maintains a local queue where the prioritisation occurs 
    
    while len(Q):
        # sort the uncomputed vertices into ascending order by estimated distance
        Q = sorted(Q, key=lambda x: x.d, reverse=False)
        # pick the best predicted v.d and move it to S
        u = Q.pop(0)
        S.append(u)

        # relax all attached edged 
        for e in G.Aj[u.id]:
            w = G.Aj[u.id][e]
            a = G.id_to_V(u)
            b = G.id_to_V(e)
            
            if b.d > a.d + w:
                b.d = a.d + w
                b.pi = a
                # update lthe local min P metric
                l_b = Q[Q.index(b.id)]
                l_b.d = a.d + w
                l_b.pi = a

    return True

print( dijkstra(G, G.id_to_V("s")) )

for v in G.V:
    print(v)

# Anti-Caching Visualiser
c_ = random.randint(0,2e9)
G.g_viz_bfs(f"{c_}.png")
display(HTML(f'<img src="{c_}.png" border="0" class="bg-primary" width="100px">'))

True
s d:[0] s[0] f[0]
a d:[3] s[0] f[0] parent[b]
b d:[2] s[0] f[0] parent[s]
c d:[7] s[0] f[0] parent[e]
d d:[8] s[0] f[0] parent[c]
e d:[5] s[0] f[0] parent[a]
Saving visualisation to 958219734.png


### Floyd-Warshall

Let $d_{ij}^k$ be the weight of the shortest path from $i$ to $j$ where all intermediate vertices can be numbered $[1..k]$ i.e. there are a maximum of $k$ vertices on the path. Hence $d_{ij}^{(0)} = w_{ij}$.

$d_{ij}^k =  \begin{cases} w_{ij}  & \text{if } k = 0 \\ 
                         min[ d_{ij}^{k-1} , d_{ik}^{k-1} + d_{kj}^{k-1}] & \text{if } k \geq 1 \end{cases}$

Where $W$ is an edge weight matrix, and there are $n$ vertices, generate the shortest path weight matrix $D^n$.

| | Floyd-Warshall (W, n) |
| -- | -- |
| 1 | $D^0$ = $W$ |
| 2 | for $k$ = 1 to $n$ |
| 3 | ¬ let $D^k$ = $d_{ij}^k$ be a new $n$ x $n$ matrix
| 4 | ¬ for $i$ = 1 to $n$ |
| 5 | ¬ ¬ ¬ for $j$ = 1 to $n$ |
| 6 | ¬ ¬ ¬ ¬  $d_{ij}^k = min[ d_{ij}^{k-1} , d_{ik}^{k-1} + d_{kj}^{k-1}]$ |
| 7 | return $D^n$ |


In [1]:
def Floyd_Warshall_diameter(L):
    # Floyd-Warshall Method
    # Create the weights matrix
    D = [[float('inf') for _ in L] for __ in L]
    n = len(L)

    for v, V in enumerate(L):
        for u in V:
            D[u][v] = 1
            D[v][u] = 1

    for k in range(n):
        for i in range(n):
            for j in range(n):
                D[i][j] = min(D[i][j], D[i][k] + D[k][j])

    max_w = max([x for d in D for x in d])
    return max_w

input_value = [[1, 2], [0, 3, 4], [0, 5, 6], [1], [1], [2], [2]]
print (Floyd_Warshall_diameter(input_value))

4


### Flow Networks and Maximum Flow

A flow network is a directed graph, where each edge as a strictly non-negative capacity. 

A flow network has two additional distinguished vertices, the *source* $s$ and the *sink* $t$. For simplicity we assume all vertices lie on a path from $s$ to $t$.

A **flow** $f(u, v)$ in a given flow network is a real valued function that satisfies the properties:
* $0 \leq f(u,v) \leq c(u, v)$ - never exceeds an edges capacity
* $\sum f(u, v) = \sum f(v, u)$ - Flow Conservation

The value of a flow equal its net sum leaving the source, $\sum f(s, v) - \sum f(v, s)$

<img src="media/flow.png" alt="drawing" width="600"/>



#### Max Flow Problem

For a given flow network with source $s$ and sink $t$, find a flow of maximum value.
* There may be multiple sources and sinks - these can be combined to create infinite capacity super source and sinks by linking them together.
* If there is an 'antiparallel edge', that being one where $(u, v) \& (v, u) \in \mathbb{E}$ split one edge into two with a central dummy vertex

#### Ford-Fulkerson Methodology

* An edge of the flow can admit an amount of additional flow = capacity - existing flow
    * This is called the *residual capacity* $c_f$
* The residual network is a graph of edges with capacities that represent how the flow can be changed
* An **augmenting path** is a simple path from $s$ to $t$ in the residual network
* The maximum amount by which we can increase the flow on an augmenting path is called its residual capacity as before

The Ford-Fulkerson method iteratively increases the value of the flow starting from 0. Each iteration 
increases the flow value in $G$ by finding an *augmenting path* in an associated *residual network*.

In order to represent a possible decrease in the positive flow $f(u, v)$ on an edge in $G$, the residual network  contains an edge $(v, u)$ with residual capacity $c_f(v, u) = f(u, v)$. that is, an edge that can admit flow in the
opposite direction, at most canceling out the flow on $(u, v)$.

Define residual capacity:

$c_f(u, v) =  \begin{cases} c(u, v) - f(u, v)  & \text{if } (u, v) \in E \\ 
                            f(u, v)            & \text{if } (v, u) \in E \\
                            0                  & \text{otherwise} \end{cases}$

As an example, if $c(u, v)$ = 16 and $f(u, v)$ = 11, then $f(u, v)$ can increase by up to $c_f(u, v)$ = 5 units before exceeding the capacity constraint on edge $(u, v)$. Alternatively, up to 11 units of flow can return from v
to u, so that $c_f(v, u) = 11$.

The max-flow min-cut theorem shows that upon termination, this process yields a maximum flow.

> Flow is maximum if and only if its residual network contains no augmenting path.

Method:
* Initialise the flow to 0
* while there is an augmenting path that exists in the residual network
    * augment the flow along that path
* return the flow

| | FORD-FULKERSON (G, s, t) |
| -- | -- |
| 1 | for each edge $\in$ $G.E$ |
| 2 | ¬ edge.f = 0 |
| 3 | While there is a path $p$ from $s$ to $t$ in the residual network: |
| 4 | ¬ residual_flow($p$) = min( residual_flow($(u, v) \forall \in p$)|
| 5 | ¬ for each edge in p: |
| 6 | ¬ ¬ if $(u,v)$ is in $G.E$|
| 7 | ¬ ¬ ¬ edge.f = $(u,v)$ + residual_flow($p$) |
| 8 | ¬ ¬ else edge.T.f = $(v,u)$ - residual_flow($p$) |
| 9 | return f |

#### Edmonds-Karp

Above, the algorithm never chooses the augmenting path with the fewest edges. It should do. 

By using BFS to find an augmenting path in the residual network, the algorithm runs in polynomial time,
independent of the maximum flow value. 

Choose the augmenting path as a shortest path from to in the residual network where
each edge has unit distance (weight).

